In [1]:
import json
import sys
import math
from tqdm import tqdm
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ph173\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ph173\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
review_path = 'before_comment.csv'
review_df = pd.read_csv(review_path,low_memory=False)

review_df = review_df.iloc[:,1:]
review_df = review_df.dropna(axis=0,how='any')
print(review_df)

# 分train和test
user_groups = dict(list(review_df.groupby('user_id')))
ct = 0
for user, df in tqdm(user_groups.items()):
    if df.shape[0] > 2:
        if ct == 0:
            all_train = df.iloc[:math.ceil(df.shape[0]*0.6), :]
            all_test = df.iloc[math.ceil(df.shape[0]*0.6):, :]
        else:
            all_train = pd.concat([all_train, df.iloc[:math.ceil(df.shape[0]*0.6), :]])
            all_test = pd.concat([all_test, df.iloc[math.ceil(df.shape[0]*0.6):, :]])
        ct += 1
all_train

all_test



                   business_id                 user_id  \
0       xXgE_rj-tK9v_MIH3mXpaQ  3W-lOOaIVQZONZP83kab6g   
1       8CSdj8LF593qwMcduzHglQ  3W-lOOaIVQZONZP83kab6g   
2       JjXEPAR_tf3VCdngVS9GMg  3W-lOOaIVQZONZP83kab6g   
3       cBHzDxKCsosdDI4UR3w-eg  3W-lOOaIVQZONZP83kab6g   
4       tU692E8N0xBQ7Ogc78gN2g  3W-lOOaIVQZONZP83kab6g   
...                        ...                     ...   
588250  wbaeORfUCcxkwdh7mNapRg  8j1OffLqKtetcAG3xv3f0Q   
588251  cJGdlK7KeE78Qa0FydikIw  VTfl9PALOCiGV8SUBpCZEQ   
588252  U7esGceQIZ_1IG6Ib0c1lQ  m0-9woIKbK-R4F8RVEe4BQ   
588253  Y4uWSRQ-GyOaxA2c-qZxWw  QTVdM8ThpshrX-8GTcdk2w   
588254  4-z563pZ4jNCVeJtPW8k_Q  EkUWydx3bJmiMWOBmdVf4Q   

                                                     text  Ratings  \
0       Reggie and company cook up the real deal at th...      4.0   
1       Convential Wisdom is that this is the best res...      3.5   
2       Finally an organic grocery store in East Nashv...      4.0   
3       This converted 

100%|█████████████████████████████████████████████████████████████████████████| 215518/215518 [14:34<00:00, 246.48it/s]


,business_id,user_id,text,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,BikeParking,Smoking,Park
108914,ORL4JE6tz3rJxVqkdKfegA,--Kwhcbkh7jxkhVVQZo2uQ,The place is absolutely huge! That's the attra...,3.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
499384,QRzaTTTE7QrqjW2GOQIrtA,--RUqD8c6ibQSh9owooVxw,Ok guys this is one of my favorite vegan spots...,4.5,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
533424,rs2OZH_vIKgW2Zk59OnZ2A,--RUqD8c6ibQSh9owooVxw,I love this place I tried the Cuban sandwich a...,4.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
190762,VAeEXLbEcI9Emt9KGYq9aA,--SQl1BQaK7Sjpt5ZZGStw,It's greasy but what do you expect. It is good...,4.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
394742,VHw4q0f8WmajCUD2wGXBDA,--SQl1BQaK7Sjpt5ZZGStw,Definitely gets busy so reserve spot in line o...,4.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
357236,d59JUaCVMZj-ZEtlWcOKZw,zzsPf8xNW11nd0B6MZqfRw,Awesome unique food and a great addition to do...,4.5,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
335253,nKJ6RnjIVfWrXO2DUrBgCw,zzz-M4QvkEpUWWPL9RTzLA,I ordered the fish tacos the other night and t...,3.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
370287,8BEIpKZu-ddcptxDbqMwLQ,zzz-M4QvkEpUWWPL9RTzLA,"I went here for the first time this morning, a...",3.5,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
390130,8WNQxb0CiROek9SW7PopDQ,zzz-M4QvkEpUWWPL9RTzLA,What a horrible experience. Either the waitres...,4.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0


In [5]:
punctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%']
corpus = []
print('a')
for sent in all_train['text']:
    sent = sent.lower()
    words = word_tokenize(sent)
    words = [word for word in words if word not in punctuations]
    stopWords = set(stopwords.words('english'))
    words = [word for word in words if word not in stopWords]
    ps = PorterStemmer()
    words = [ps.stem(word) for word in words]
    corpus.append(' '.join(words))
corpus

a


["went nice lunch price gaylord stay n't friendli servic excel andrea friendli attent earn everi cent tip happi give glass never empti 7oz sirloin broccoli sweet tea steak tender season well excel broccoli though look fairli wilt n't great problem ask might want onion cook steak n't told littl sprinkl caramel onion would add 3 bill would much rather spent money toward dessert",
 "ate dinner tonight servic pretti fast nice fact might tri littl hard nice flirti one-piec fri catfish mash potato fri okra catfish pretti good though hot sauc help potato gravi damn good even chocol peanut butter pie n't bad includ sweet tea bill 21 decor ... interest taxidermi place bit creepi went hunt camp theme place ... guess overal good meal",
 "best japanes food 've nashvil sinc tast tokyo close tri yaki soba noodl come time week amaz food",
 "ok best chines food 've nashvil 10 year realli yummy.i veget lo mein pretti good spring roll crab ragoon scallion pancak amaz would also recommend fri rice",
 'lo

In [6]:
cntVector = TfidfVectorizer(stop_words='english',decode_error='ignore',max_features=300,max_df=0.5).fit(corpus)
cntTf = cntVector.transform(corpus)

print(cntTf.todense())

print(cntVector.vocabulary_)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.31544369]]
{'went': 290, 'nice': 179, 'lunch': 163, 'price': 207, 'stay': 253, 'friendli': 110, 'servic': 236, 'excel': 90, 'attent': 16, 'everi': 87, 'happi': 122, 'sweet': 260, 'tea': 266, 'steak': 254, 'season': 230, 'look': 160, 'great': 115, 'ask': 14, 'want': 286, 'cook': 56, 'told': 274, 'littl': 154, 'money': 174, 'dessert': 69, 'dinner': 72, 'pretti': 206, 'fast': 98, 'tri': 277, 'hard': 123, 'fri': 108, 'potato': 205, 'good': 113, 'hot': 129, 'sauc': 228, 'help': 124, 'bad': 20, 'includ': 138, 'decor': 66, 'place': 199, 'bit': 32, 'overal': 189, 'meal

In [7]:
key_list = []
value_list = []
for key,value in cntVector.vocabulary_.items():
    key_list.append(key)
    value_list.append(value)

for i in range(300):
    for value in value_list:
        if i == value:
            index = value_list.index(value)
    name = key_list[index]
    all_train[name] = cntTf.todense()[:,i]
all_train

C:\Users\ph173\AppData\Local\Temp/ipykernel_39600/2244961617.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  all_train[name] = cntTf.todense()[:,i]


,business_id,user_id,text,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,...,went,wife,wine,wish,wo,wonder,work,worth,wrong,year
108712,arQfMJal1tl67Z96ROgPFg,--Kwhcbkh7jxkhVVQZo2uQ,Went for a nice lunch because prices in the Ga...,2.5,1.0,0.0,0.0,1.0,1.0,0.0,...,0.148474,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
108739,tIvfmgT1qMeAEQf8CI5fPQ,--Kwhcbkh7jxkhVVQZo2uQ,Ate dinner there tonight. Service was pretty f...,3.5,1.0,0.0,0.0,0.0,0.0,1.0,...,0.142877,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
300758,65_MIvrauhGs_jssN4-uUQ,--RUqD8c6ibQSh9owooVxw,This is the best Japanese food I've had in Nas...,4.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
318212,xxx_0Ol7lbK0hVzv7x0-Yg,--RUqD8c6ibQSh9owooVxw,Ok so this is the best Chinese food I've had i...,3.5,1.0,0.0,1.0,1.0,0.0,1.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.294119
331053,UesqLOoyOGPQsonHpMb59g,--RUqD8c6ibQSh9owooVxw,I love this place I got my lashes done by Nina...,3.5,0.0,0.0,0.0,0.0,0.0,1.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87906,hn3Rg2JrhQoDJBEhrpuwWg,zzz-M4QvkEpUWWPL9RTzLA,We have been twice and LOVED it the first time...,3.5,1.0,1.0,0.0,1.0,1.0,0.0,...,0.105121,0.0,0.300430,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
153153,Y989Hsw30AYUMcyNjBz9iQ,zzz-M4QvkEpUWWPL9RTzLA,We went here last Friday night with friends an...,3.5,1.0,1.0,0.0,0.0,1.0,1.0,...,0.125060,0.0,0.357413,0.0,0.0,0.160329,0.0,0.0,0.0,0.000000
153155,dNmqRPHBys4dWH4nReO7MA,zzz-M4QvkEpUWWPL9RTzLA,"Based on the website, I was expecting somethin...",3.5,1.0,1.0,1.0,0.0,1.0,1.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
153392,qO-r-NipYXcuVXjIbaglEQ,zzz-M4QvkEpUWWPL9RTzLA,I got my haircut on Saturday and I've had at l...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.225635,0.0,0.0,0.0,0.000000


In [8]:
all_train = all_train.drop(columns=['text'])
all_train.to_csv('tfidf_train.csv')

In [17]:
expanded = []

for key,value in cntVector.vocabulary_.items():
    key_list.append(key)
    value_list.append(value)

for i in range(300):
    for value in value_list:
        if i == value:
            index = value_list.index(value)
    name = key_list[index]
    all_test[name] = 0.0

temp = all_test.iloc[:,1:]
for index, row in all_test.iterrows():
    if True:
        temp = all_train.loc[(all_train['user_id'] == row['user_id'])]
#         print(temp)
        sum_list = [0 for i in range(300)]
        num_rows = 0
        for i,p in temp.iterrows():
            if i < index:
                for num in range(300):
#                 print(p[num + 12])
                    sum_list[num] += p[num + 12]
                num_rows += 1
#         print(np.array(sum_list) / num_rows)
            
        for x in range(300):
            all_test.at[index,key_list[x]]= sum_list[value_list[x]] / num_rows
                
#             print(np.array(sum_list)[num] / num_rows)
#             print(row)
#         expanded.append(row['user_id'])
all_test

,business_id,user_id,text,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,...,went,wife,wine,wish,wo,wonder,work,worth,wrong,year
108914,ORL4JE6tz3rJxVqkdKfegA,--Kwhcbkh7jxkhVVQZo2uQ,The place is absolutely huge! That's the attra...,3.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.145676,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
499384,QRzaTTTE7QrqjW2GOQIrtA,--RUqD8c6ibQSh9owooVxw,Ok guys this is one of my favorite vegan spots...,4.5,1.0,1.0,1.0,1.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.073530
533424,rs2OZH_vIKgW2Zk59OnZ2A,--RUqD8c6ibQSh9owooVxw,I love this place I tried the Cuban sandwich a...,4.5,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.073530
190762,VAeEXLbEcI9Emt9KGYq9aA,--SQl1BQaK7Sjpt5ZZGStw,It's greasy but what do you expect. It is good...,4.0,1.0,1.0,0.0,1.0,1.0,0.0,...,0.000000,0.0,0.000000,0.077334,0.000000,0.000000,0.059389,0.259429,0.0,0.066885
394742,VHw4q0f8WmajCUD2wGXBDA,--SQl1BQaK7Sjpt5ZZGStw,Definitely gets busy so reserve spot in line o...,4.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.077334,0.000000,0.000000,0.059389,0.259429,0.0,0.066885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357236,d59JUaCVMZj-ZEtlWcOKZw,zzsPf8xNW11nd0B6MZqfRw,Awesome unique food and a great addition to do...,4.5,0.0,1.0,1.0,0.0,0.0,1.0,...,0.039695,0.0,0.000000,0.000000,0.000000,0.000000,0.015309,0.000000,0.0,0.019372
335253,nKJ6RnjIVfWrXO2DUrBgCw,zzz-M4QvkEpUWWPL9RTzLA,I ordered the fish tacos the other night and t...,3.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.038363,0.0,0.109641,0.000000,0.024983,0.064327,0.000000,0.000000,0.0,0.052574
370287,8BEIpKZu-ddcptxDbqMwLQ,zzz-M4QvkEpUWWPL9RTzLA,"I went here for the first time this morning, a...",3.5,1.0,1.0,1.0,0.0,0.0,0.0,...,0.038363,0.0,0.109641,0.000000,0.024983,0.064327,0.000000,0.000000,0.0,0.052574
390130,8WNQxb0CiROek9SW7PopDQ,zzz-M4QvkEpUWWPL9RTzLA,What a horrible experience. Either the waitres...,4.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.038363,0.0,0.109641,0.000000,0.024983,0.064327,0.000000,0.000000,0.0,0.052574


In [18]:
all_test = all_test.drop(columns=['text'])
all_test.to_csv('tfidf_test.csv')